Homework 3: Search for Movie Plots  
DSBA 6188  
Eric Phann



# Retrieve & Re-Rank - Search for Wikipedia Movie Plots

This notebook demonstrates the Retrieve & Re-Rank Setup.

You can input a query or a question. The script then uses semantic search
to find relevant movies in Wikipedia Movie Plots (first 1000 movies, before 1920).

For semantic search, we use `nq-distilbert-base-v1` and retrieve 5 potential movies that answer the input query.

Next, we use a more powerful CrossEncoder (`cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')`) that
scores the query and all retrieved movies for their relevancy. The cross-encoder further boost the performance,
especially when you search over a corpus for which the bi-encoder was not trained for.


## Install/import dependencies

In [ ]:
%%capture
!pip install -U sentence-transformers rank_bm25

In [ ]:
%%capture
!pip install datasets

In [7]:
%%capture
!pip install session-info

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np

In [3]:
from scipy.special import softmax

In [ ]:
from datasets import load_dataset

In [8]:
import session_info
session_info.show()

## Import data

As a dataset, we use WIkipedia Movie Plots (first 1000 movies from 1920 or before). We can import this using Hugging Face `datasets`.

In [ ]:
# import data
ds = load_dataset("Coder-Dragon/wikipedia-movies", split='train[:1000]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# see an example of the data
ds[0]

{'Release Year': 1901,
 'Title': 'Kansas Saloon Smashers',
 'Origin/Ethnicity': 'American',
 'Director': 'Unknown',
 'Cast': None,
 'Genre': 'unknown',
 'Wiki Page': 'https://en.wikipedia.org/wiki/Kansas_Saloon_Smashers',
 'Plot': "A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]",
 'Image': 'upload.wikimedia.org/wikipedia/commons/2/2d/KansasSaloonSmashers1901.jpg'}

## Create corpus embeddings (Encode)

In [ ]:
# We use the Bi-Encoder to encode all movies, so that we can use it with semantic search
model_name = 'nq-distilbert-base-v1'
bi_encoder = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/540 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# create movies[] as [title: plot]
movies = []
for title, plot in zip(ds['Title'], ds['Plot']):
  movies.append(": ".join([title, plot]))

# examine movies[]
print(len(movies), "\n")
movies[:3]

1000 



["Kansas Saloon Smashers: A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]",
 "Love by the Light of the Moon: The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better."

In [ ]:
# create corpus embeddings
corpus_embeddings = bi_encoder.encode(movies, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
corpus_embeddings[:5]

tensor([[ 0.4239,  0.2595, -0.6739,  ..., -0.4917, -0.8164, -0.4017],
        [ 0.4389,  0.4659,  0.7291,  ..., -0.3748, -0.1628,  0.0937],
        [ 0.5075, -0.0324, -0.1718,  ...,  0.1030,  0.1665,  0.2042],
        [-0.0996,  0.1081, -0.2503,  ...,  0.0924, -0.7941,  0.7857],
        [-0.2011, -0.1119, -0.9650,  ...,  0.0308, -0.3965, -0.7012]])

## Tokenize corpus and generate BM25 scores

In [ ]:
# We also compare the results to lexical search (keyword search). Here, we use
# the BM25 algorithm which is implemented in the rank_bm25 package.


# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc


tokenized_corpus = []
for movie in tqdm(movies):
    tokenized_corpus.append(bm25_tokenizer(movie))

bm25 = BM25Okapi(tokenized_corpus)


  0%|          | 0/1000 [00:00<?, ?it/s]

## Search function

Output top 5:


*   BM25: sophisticated extension of TF-IDF (lexical/keyword search)

*   bi-encoder: `nq-distilbert-base-v1` (semantic search using cosine similarity w/ embeddings)
*   cross-encoder: `ms-marco-MiniLM-L-6-v2` (rerank semantic search results)



In [ ]:
# We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# This function will search all wikipedia articles for passages that answer the query
# and return top_k results
top_k = 5

def search(query):
    print("Query:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print("Top-5 lexical search (BM25) hits")
    for hit in bm25_hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['score'], movies[hit['corpus_id']]))

    ##### Semantic Search #####
    # Encode the query using the bi-encoder and find potentially relevant movies
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, movies[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)
    # run this through a softmax to get probabilites
    cross_scores = softmax(cross_scores)
    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-5 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['score'], movies[hit['corpus_id']]))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-5 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], movies[hit['corpus_id']]))


## Search Examples & Metrics

### Evaluation Metrics

*   Recall@1
  * $Recall@k = \frac{true\,positives@k}{(true\,positives@k + false\,negatives@k)}$
  * How many actual relevent results shown from top k over all actual relevent results from query?
  * In our case, is our 1 "ground truth movie" the first movie returned? (yes=1, no=0)
*   Mean Reciprocal Rank (MRR)
  * $MRR = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{rank_i}$
  * Averaging the highest ranks across queries


### Example 1

In [ ]:
search("Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions")

Query: Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions
Top-5 lexical search (BM25) hits
	9.241	I Do: The Boy meets and marries The Girl. A year later, the two walk down the street with a baby carriage carrying a bottle instead of a baby when they run into The Girl's brother who asks the couple to do him a favor and babysit his children. They accept and the remainder of the short consists of gags showcasing the difficulties of babysitting children. At the very end, The Boy discovers some knitted baby clothes in a drawer (implying that The Girl is pregnant).
	8.233	Uncharted Seas: As described in a film magazine,[3] after her drunken husband Tom Eastman (Gerard) brings home three cabaret women, Lucretia (Lake) can no longer bear the abuse and turns to arctic explorer Frank Underwood (Valentino), who has long loved her and promised to come whenever she needs his help. Urging her husband to become a man and do something worth wile, Lucretia goes with 

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Ground Truth Movie = 'Nanook of the North'  

BM25:

*   Recall@1 = 0
*   Reciprocal Rank = 0

Reranker:

*   Recall@1 = 1
*   Reciprocal Rank = 1

### Example 2

In [ ]:
search("Western romance")

Ground Truth Movie = 'The Lucky Horseshoe'  

BM25:

*   Recall@1 = 0
*   Reciprocal Rank = 0

Reranker:

*   Recall@1 = 0
*   Reciprocal Rank = 0

### Example 3

In [ ]:
search("Silent film about a Parisian star moving to Egypt, leaving her husband \
for a baron, and later reconciling after finding her family in poverty in Cairo.")

Ground Truth Movie = 'Sahara'  

BM25:

*   Recall@1 = 1
*   Reciprocal Rank = 1

Reranker:

*   Recall@1 = 1
*   Reciprocal Rank = 1

### Example 4

In [ ]:
search("Comedy film, office disguises, boss's daughter, elopement.")

Ground Truth Movie = 'Ask Father'  

BM25:

*   Recall@1 = 0
*   Reciprocal Rank = 1/3

Reranker:

*   Recall@1 = 0
*   Reciprocal Rank = 0

### Example 5

In [ ]:
search("Lost film, Cleopatra charms Caesar, plots world rule, treasures from \
mummy, revels with Antony, tragic end with serpent in Alexandria.")

Ground Truth Movie = 'Cleopatra'  

BM25:

*   Recall@1 = 1
*   Reciprocal Rank = 1

Reranker:

*   Recall@1 = 1
*   Reciprocal Rank = 1

### Example 6

In [ ]:
search("Denis Gage Deane-Tanner")

Ground Truth Movie = 'Captain Alvarez'  

BM25:

*   Recall@1 = 1
*   Reciprocal Rank = 1

Reranker:

*   Recall@1 = 0
*   Reciprocal Rank = 0

## Results

BM25:

*   3 out of 6 (0.50) queries returned the ground truth as the top result (Average Recall@1)
*   Mean Reciprocal Rank (MRR) = (1 + 1 + 1 + 1/3 + 0)/6 = 10/18 = 0.55

Reranker:


*   3 out of 6 (0.50) queries returned the ground truth as the top result (Average Recall@1)
*   Mean Reciprocal Rank (MRR) = (1 + 1 + 1 + 0 + 0 + 0)/6 = 1/2 = 0.50



## Analysis

BM25 and Reranked perform similarly across both metrics, with BM25 performing 0.05 better in MRR than Reranker.  

However, it is worth noting that one method might perform better with certain queries than the other:


*   BM25 hits 'Captain Alvarez' with query "Denis Gage Deane-Tanner" while Reranker doesn't (Example 6)
*   BM25 hits 'Ask Father' 3rd with query "Comedy film, office disguises, boss's daughter, elopement." while Reranker doesn't (Example 4)
*   Reranker hits 'Nanook of the North' with query "Documentaries showcasing indigenous peoples' survival and daily life in Arctic regions" (Example 1)  

This suggests that there is some nuance between the two methods (lexical vs. semantic search) for this use case. It seems that BM25 can better hit with shorter queries.  

It is also worth noting that Example 2 is probably a dud: it is too vague and short to return the 1 ground truth movie. Neither method can read the user's mind.

